In [ ]:
!pip install transformers
!pip install sentence_transformers
!pip install datasets
!pip install accelerate
!pip install sentencepiece


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.9 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=dfbcd191c616b7bcd38b0f2d9fec95ad9f9a8344fe1b4688a96556db518e667b
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.2 MB/s eta 0:00:00


In [1]:
import pandas as pd
from transformers import AutoModelForSeq2SeqLM ,TrainingArguments, AutoTokenizer , DataCollatorForSeq2Seq,Seq2SeqTrainer , Trainer , MarianMTModel, MarianTokenizer
from datasets import Dataset , load_dataset
import tensorflow as tf
import json
import ast

c:\Users\7fno\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


c:\Users\7fno\AppData\Local\Programs\Python\Python311\Lib\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [ ]:
tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ar-en")
model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-ar-en")

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/917k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer , model = model,return_tensors=  'tf')

In [ ]:
raw_datasets = load_dataset("cfilt/iitb-english-hindi",split = 'train[:100]')
raw_datasets

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1659083 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/520 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2507 [00:00<?, ? examples/s]

Dataset({
    features: ['translation'],
    num_rows: 100
})

In [ ]:
raw_datasets = load_dataset("cfilt/iitb-english-hindi",split = 'train[:100]')
raw_datasets

Dataset({
    features: ['translation'],
    num_rows: 100
})

In [ ]:
max_input =256
max_output= 256
def process(example):
  inputs = [ss['en']for ss in example['translation']]
  target = [ss['hi']for ss in example['translation']]
  model_inputs = tokenizer(inputs , max_length=256 , truncation=True)
  with tokenizer.as_target_tokenizer():
   labels = tokenizer(target , max_length=max_output , truncation=True)
  model_inputs['labels'] = labels ['input_ids']
  return model_inputs
# process(raw_datasets)
tokenized_data = raw_datasets.map(process, batched = True)
tokenized_data

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Dataset({
    features: ['translation', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 100
})

In [ ]:
tokenized_data

Dataset({
    features: ['translation', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 100
})

In [ ]:
tokenizer.decode(tokenized_data['input_ids'][0])

'Give your application an accessibility workout</s>'

In [ ]:
data = pd.read_csv(r'/content/Book12.csv')

In [ ]:
# def format(ss, nn):
#     return f"{repr('en')}" +':'+ f'{repr(ss)}' ',' +f"{repr('ar')}"+':'+f'{repr(nn)}'

# # Apply the function to create a new column 'all'
# data['Translation'] = data.apply(lambda row: format(row['English'], row['عربي']), axis=1)
data = data.drop(['Translation'],axis=1)


In [ ]:
data

,عربي,English
0,شهد العالم الاسلامي أواخر القرن الخامس عشر و ب...,The Islamic world witnessed an intense conflic...
1,دولة المماليك قامت في مصر و الشام و الحجاز و ج...,"The Mamluk state was established in Egypt, the..."
2,وتحطم الاسطول المملوكي عام 1509م في معركة ديو ...,", and the destruction of the Mamluk fleet in 1..."
3,الدولة العثمانية مع بداية القرن السادس عشر الم...,The Ottoman Empire: At the beginning of the si...
4,الدولة الصفوية و بعد ان تمكن السلطان سليم الاو...,The Safavid state. After Sultan Selim I was ab...
5,اتجه العثمانيين الي الي مصر و الشام بسبب رغبة ...,The Ottomans turned to Egypt and the Levant be...
6,في ذلك الوقت محفوق بالمخاطر بسبب تفوقهم حضاريا...,at that time was fraught with danger because o...
7,استيلاء العثمانيين علي مصر و الشام في مرج دابق...,The Ottomans took control of Egypt and the Lev...
8,التقي العثمانيون بجيوش طومان باي عند الريدانية...,The Ottomans met the armies of Tuman Bey at Al...
9,تم هزيمة الجيوش المملوكية بسبب التفوق العسكري ...,The Mamluk armies were defeated due to the mil...


In [ ]:
# data['Translation'][0]

In [ ]:
data= Dataset.from_pandas(data)
data = data.train_test_split(.1)

In [ ]:
# data_strings = data['train']['Translation'][:]
#  # Assuming the translation is the first element in the list
# key_value_pairs = []
# for data_string in data_strings:
#     pairs = [pair.strip() for pair in data_string.strip("[]").split(',')]
#     key_value_pairs.extend(pairs)

# # Convert key-value pairs to a valid JSON string
# json_string = '{{{}}}'.format(','.join(key_value_pairs))

# # Load the JSON string into a Python dictionary
# data_dict = json.loads(json_string)


In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['عربي', 'English'],
        num_rows: 17
    })
    test: Dataset({
        features: ['عربي', 'English'],
        num_rows: 2
    })
})

In [ ]:
tokenizer('اهلا انا محمد')

{'input_ids': [17037, 30, 842, 7434, 0], 'attention_mask': [1, 1, 1, 1, 1]}

In [ ]:
data['train'][:1]

{'عربي': ['شهد العالم الاسلامي أواخر القرن الخامس عشر و بداية القرن السادس عشر صراعا حادا بس ثلاث قوي هي دولة المماليك  و الدولة الصفوية و الدولة العثمانية .'],
 'English': ['The Islamic world witnessed an intense conflict in the late fifteenth century and the beginning of the sixteenth century, but there were three powerful forces: the Mamluk state, the Safavid state, and the Ottoman state.']}

In [ ]:
data['train'][0]

{'عربي': 'شهد العالم الاسلامي أواخر القرن الخامس عشر و بداية القرن السادس عشر صراعا حادا بس ثلاث قوي هي دولة المماليك  و الدولة الصفوية و الدولة العثمانية .',
 'English': 'The Islamic world witnessed an intense conflict in the late fifteenth century and the beginning of the sixteenth century, but there were three powerful forces: the Mamluk state, the Safavid state, and the Ottoman state.'}

In [ ]:
len(data['train'])

17

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['عربي', 'English'],
        num_rows: 17
    })
    test: Dataset({
        features: ['عربي', 'English'],
        num_rows: 2
    })
})

In [ ]:
# answer_str = data['train']['Translation'][12]
# dataes = json.loads(answer_str.replace("'", "\""))
# # data['train']['Translation'][0]['en']


In [ ]:
max_input =256
max_output= 256
def process(example):
  inputs = [ss['عربي']for ss in example]
  target = [ss['English']for ss in example]
  model_inputs = tokenizer(inputs , max_length=256 , truncation=True)
  with tokenizer.as_target_tokenizer():
   labels = tokenizer(target , max_length=max_output , truncation=True)
  model_inputs['labels'] = labels ['input_ids']
  return model_inputs


In [ ]:
process(data['train'])
print()

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['عربي', 'English'],
        num_rows: 17
    })
    test: Dataset({
        features: ['عربي', 'English'],
        num_rows: 2
    })
})

In [ ]:
raw_datasets

Dataset({
    features: ['translation'],
    num_rows: 100
})

In [ ]:
max_input =256
max_output= 256
def process(example):
  # print(example['عربي'][0])
  inputs = [ss for ss in example['عربي']]
  target = [ss for ss in example['English']]
  model_inputs = tokenizer(inputs , max_length=256 , truncation=True)
  with tokenizer.as_target_tokenizer():
   labels = tokenizer(target , max_length=max_output , truncation=True)
  model_inputs['labels'] = labels ['input_ids']
  return model_inputs

In [ ]:
tokenized_data = data.map(process, batched=True,remove_columns=data['train'].column_names)

Map:   0%|          | 0/17 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [ ]:
tokenized_data

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 17
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2
    })
})

In [ ]:
tokenizer.decode(tokenized_data['train']['input_ids'][3])

'اتجه العثمانيين الي الي مصر و الشام بسبب رغبة السلطان سليم الاول في توسيع ممتلكاته في الشرق عن طريق الاستيلاء علي مصر التي تمثل قلب العالم الاسلامي و ايضا بسبب ادراك السلطان سليم الاول ان الصدام مع الدولة الاوروبية</s>'

In [ ]:
# process(data['train'])

In [ ]:
# dataes = []
# for i in range(17):
#   answer_str = data['train']['Translation'][i]
#   datas = json.loads(answer_str.replace("'", "\""))
#   dataes.append(datas)

In [ ]:
# dataes[0]['en']

In [ ]:
data_collator = DataCollatorForSeq2Seq(model = model  ,tokenizer = tokenizer)

In [ ]:
tokenized_data['train']

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 17
})

In [ ]:
training_args = TrainingArguments('finetuned_model',learning_rate = 2e-5, per_device_train_batch_size=32,
                                      num_train_epochs  = 20 ,
                                      evaluation_strategy = 'epoch')

In [ ]:
trainer = Trainer(model=model, args=training_args
                         ,train_dataset = tokenized_data['train']
                         ,eval_dataset = tokenized_data['test']
                         ,data_collator = data_collator
                         ,tokenizer = tokenizer)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,1.610756
2,No log,1.527933
3,No log,1.443931
4,No log,1.374197
5,No log,1.319512
6,No log,1.275993
7,No log,1.238244
8,No log,1.205847
9,No log,1.178291
10,No log,1.154274


TrainOutput(global_step=20, training_loss=0.85272216796875, metrics={'train_runtime': 521.5018, 'train_samples_per_second': 0.652, 'train_steps_per_second': 0.038, 'total_flos': 8193870397440.0, 'train_loss': 0.85272216796875, 'epoch': 20.0})

In [ ]:
trainer.save_model('/content/drive/MyDrive/trained_models/translation_model.h5')

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained('my_translation_model')
tokenizer = AutoModelForSeq2SeqLM.from_pretrained("my_translation_model")



In [ ]:
model_name = 'my_translation_model'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

input_text = '''لماذا عانت الدولة الصفوية'''

# Tokenize input text
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# Generate translations
output = model.generate(input_ids)

# Decode the generated output
translated_text = tokenizer.decode(output[0], skip_special_tokens=True)

In [ ]:
print(translated_text)

Why did the Safavid state suffer?


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model.save_pretrained('/content/drive/MyDrive/trained_models')